In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [8]:
# Setup
df = pd.read_csv('Mental_Health_dataset1.csv')
df.drop('SocialWeakness.1', axis=1, inplace=True)
DropCountries = ['Bosnia and Herzegovina', 'Brazil', 'Colombia', 'Costa Rica', 'Croatia', 'Czech Republic', 'Denmark', 'Finland', 'France', 'Georgia', 'Germany', 'Greece', 'Israel', 'Italy', 'Mexico', 'Moldova', 'Nigeria', 'Philippines', 'Portugal', 'Russia', 'Singapore', 'Switzerland', 'Thailand']
df = df[~df['Country'].isin(DropCountries)]
df.dropna()
df.describe()

,Gender,Country,Occupation,SelfEmployed,FamilyHistory,Treatment,DaysIndoors,HabitsChange,MentalHealthHistory,IncreasingStress,MoodSwings,SocialWeakness,CopingStruggles,WorkInterest,MentalHealthInterview,CareOptions
count,240534,240534,240534,236178,240534,240534,240534,240534,240534,240534,240534,240534,240534,240534,240534,240534
unique,2,12,5,2,2,2,5,3,3,3,3,3,2,3,3,3
top,Male,USA,Housewife,No,No,Yes,1-14 days,Yes,No,Maybe,Medium,Maybe,No,No,No,No
freq,209820,154478,55275,213894,146505,123306,52952,89865,86456,83346,82982,85797,127969,86890,191273,97680


In [ ]:
# Readjust Values from DataFrame


In [13]:
# Add Mental Health Problems Binary Column
# If Treatment or MentalHealthHistory or MentalHealthInterview = 1, Then Mental Health Problems = 1
df['MentalHealthIssues'] = np.where((df['MentalHealthHistory'] == 'Yes') | (df['MentalHealthInterview'] == 'Yes') | (df['Treatment'] == 'Yes'), 1, 0)
df.describe()

,MentalHealthIssues
count,240534.000000
mean,0.674217
std,0.468668
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [ ]:
# Train / Test
Independent_Variables = ['Occupation', 'Gender', 'Country', 'SelfEmployed', 'FamilyHistory', 'DaysIndoors', 'HabitsChange', 'IncreasingStress', 'MoodSwings'].values.reshape(-1,9) 
Dependent_Variable = ['MentalHealthIssues'].values.reshape(-1,1)